In [0]:
import os
ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
SECRET_KEY = os.getenv('AWS_SECRET_KEY')
AWS_BUCKET_SOURCE = 'aida-project'
MOUNT_SOURCE = '/mnt/data_source'

dbutils.fs.mount(f's3a://{ACCESS_KEY}:{SECRET_KEY.replace("/", "%2F")}@{AWS_BUCKET_SOURCE}', MOUNT_SOURCE)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2238110042734527> in <module> 
 5 MOUNT_SOURCE = '/mnt/data_source' 
 6 
 ----> 7 dbutils . fs . mount ( f's3a://{ACCESS_KEY}:{SECRET_KEY.replace("/", "%2F")}@{AWS_BUCKET_SOURCE}' , MOUNT_SOURCE ) 

 /local_disk0/tmp/1605521044100-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o243.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data_source; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data_source
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:471)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data_source
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:220)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:322)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:216)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:77)
	at com.databricks.backend.daemon.data.server.handler.CEMountHandler.receive(MountHandler.scala:122)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:354)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:322)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.Usag

In [0]:
display(dbutils.fs.ls(f'dbfs:{MOUNT_SOURCE}/TSV'))

In [0]:
tables = {}
for file in dbutils.fs.ls(f'{MOUNT_SOURCE}/TSV'):
  table = {'name.basics.tsv':'names', 'title.basics.tsv':'titles', 'title.akas.tsv':'akas',
    'title.principals.tsv':'principals', 'title.ratings.tsv':'ratings'}[file.name]
  print('Loading', file.path, "-> tables['" + table + "']")
  tables[table] = spark.read.csv(file.path, header=True, inferSchema=True, sep='\t', nullValue=r'\N')
  print(table, f'({tables[table].count():,} rows)', end=': '); tables[table].printSchema()

Loading dbfs:/mnt/data_source/TSV/name.basics.tsv -> tables['names']
names (9,711,022 rows): root
-- nconst: string (nullable = true)
-- primaryName: string (nullable = true)
-- birthYear: integer (nullable = true)
-- deathYear: integer (nullable = true)
-- primaryProfession: string (nullable = true)
-- knownForTitles: string (nullable = true)

Loading dbfs:/mnt/data_source/TSV/title.akas.tsv -> tables['akas']
akas (19,549,634 rows): root
-- titleId: string (nullable = true)
-- ordering: integer (nullable = true)
-- title: string (nullable = true)
-- region: string (nullable = true)
-- language: string (nullable = true)
-- types: string (nullable = true)
-- attributes: string (nullable = true)
-- isOriginalTitle: integer (nullable = true)

Loading dbfs:/mnt/data_source/TSV/title.basics.tsv -> tables['titles']
titles (6,326,545 rows): root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: integer (nullable = true)
-- endYear: integer (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)

Loading dbfs:/mnt/data_source/TSV/title.principals.tsv -> tables['principals']
principals (36,499,704 rows): root
-- tconst: string (nullable = true)
-- ordering: integer (nullable = true)
-- nconst: string (nullable = true)
-- category: string (nullable = true)
-- job: string (nullable = true)
-- characters: string (nullable = true)

Loading dbfs:/mnt/data_source/TSV/title.ratings.tsv -> tables['ratings']
ratings (993,821 rows): root
-- tconst: string (nullable = true)
-- averageRating: double (nullable = true)
-- numVotes: integer (nullable = true)

In [0]:
# Julian's work goes here:
# Just in case it is needed later - the file is located here: s3://sjf-project/julian_genres.tsv
# I will use the 'julian' object defined before...
julian = tables['titles'].select('tconst, titleType, startYear, endYear, genres'.split(', '))
julian.printSchema()

#from pyspark.sql.functions import col, asc


root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- startYear: integer (nullable = true)
-- endYear: integer (nullable = true)
-- genres: string (nullable = true)

In [0]:
#julian.show(500)

nogenre = julian.filter(julian.genres.isNull())#.sort(julian.tconst)
nogenre.show()

+---------+---------+---------+-------+------+
 tconst|titleType|startYear|endYear|genres|
+---------+---------+---------+-------+------+
tt0000502| movie| 1905| null| null|
tt0000812| movie| 1909| null| null|
tt0000814| movie| 1909| null| null|
tt0000838| movie| 1909| null| null|
tt0000842| movie| 1909| null| null|
tt0000846| movie| 1909| null| null|
tt0000850| movie| 1909| null| null|
tt0000859| movie| 1909| null| null|
tt0000862| movie| 1909| null| null|
tt0000867| movie| 1909| null| null|
tt0000868| movie| 1909| null| null|
tt0000869| movie| 1909| null| null|
tt0000879| movie| 1909| null| null|
tt0000891| movie| 1909| null| null|
tt0000947| movie| 1909| null| null|
tt0000959| movie| 1909| null| null|
tt0000984| movie| 1909| null| null|
tt0000992| movie| 1909| null| null|
tt0001007| movie| 1909| null| null|
tt0001010| movie| 1909| null| null|
+---------+---------+---------+-------+------+
only showing top 20 rows

In [0]:
nogenre.count()

Out[10]: 501660

In [0]:
from pyspark.sql.functions import * # import all sql functionsfor pyspark



1
[[Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Animation,Short']), Row(split(genres, [,], 1)=['Animation,Comedy,Romance']), Row(split(genres, [,], 1)=['Animation,Short']), Row(split(genres, [,], 1)=['Comedy,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short,Sport']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Romance']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Action,Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Comedy,Short']), Row(split(genres, [,], 1)=['Animation,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Comedy,Short']), Row(split(genres, [,], 1)=['Documentary,Short,Sport']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['News,Short']), Row(split(genres, [,], 1)=['News,Short,Sport']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Comedy,Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Comedy,Short']), Row(split(genres, [,], 1)=['Drama,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short,Sport']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Action,Comedy,Documentary']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short,Sport']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Fantasy,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Short,Sport']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Horror,Short']), Row(split(genres, [,], 1)=['Drama,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Comedy,Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Documentary,Short']), Row(split(genres, [,], 1)=['Doc

In [0]:
julian.select(split(julian.genres, '[,]', -1)).collect()

Out[26]: [Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Animation', 'Short']),
 Row(split(genres, [,], -1)=['Animation', 'Comedy', 'Romance']),
 Row(split(genres, [,], -1)=['Animation', 'Short']),
 Row(split(genres, [,], -1)=['Comedy', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short', 'Sport']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Romance']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Action', 'Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Comedy', 'Short']),
 Row(split(genres, [,], -1)=['Animation', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Comedy', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short', 'Sport']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['News', 'Short']),
 Row(split(genres, [,], -1)=['News', 'Short', 'Sport']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Comedy', 'Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Comedy', 'Short']),
 Row(split(genres, [,], -1)=['Drama', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short', 'Sport']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Action', 'Comedy', 'Documentary']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short', 'Sport']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Fantasy', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Short', 'Sport']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Horror', 'Short']),
 Row(split(genres, [,], -1)=['Drama', 'Short']),
 Row(split(genres, [,], -1)=['Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentary', 'Short']),
 Row(split(genres, [,], -1)=['Documentar